In [1]:
import numpy as np
import pandas as pd
import random
from typing import Union, Callable

In [2]:
class MyLogReg():
    """
    Линейная регрессия

    Гиперпараметры:
    ***************
    n_iter : int, optional
        Количество шагов градиентного спуска (default: 100)
    learning_rate : Union[float, Callable], optional
        Коэффициент скорости обучения градиентного спуска.
        Если на вход пришла lambda-функция, то learning_rate вычисляется 
        на каждом шаге на основе переданной функцией (default: 0.01)
    weights : np.ndarray, optional
        Веса модели (default: None)
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: mae, mse, rmse, mape, r2 (default: None_
    reg : str, optional
        Вид регуляризации. Принимает одно из следующих значений: l1, l2, elasticnet (default: None)
    l1_coef : float, optional
        Коэффициент L1 регуляризации. Принимает значения от 0.0 до 1.0 (default: 0)
    l2_coef : float, optional
        Коэффициент L2 регуляризации. Принимает значения от 0.0 до 1.0 (default: 0)
    sgd_sample : Union[int, float], optional
        Количество образцов, которое будет использоваться на каждой итерации обучения.
        Может принимать целые числа, либо дробные от 0.0 до 1.0 (default: None)
    random_state : int, optional
        Сид для воспроизводимости результата (default: 42)
    """

    def __init__(self,
                n_iter: int = 100,
                learning_rate: float = 0.01,
                weights: np.ndarray = None,
                metric: str = None,
                reg: str = None,
                l1_coef: float = 0,
                l2_coef: float = 0,
                sgd_sample: float = None,
                random_state: int = 42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.metric = metric
        self.score = None
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        self.random_state = random_state


    def __str__(self):
        params = [f'{k}={v}' for k,v in self.__dict__.items()]
        return 'MyLineReg class: ' + ', '.join(params)


    __repr__ = __str__

    @staticmethod
    def mae(y_true: pd.Series, y_pred: pd.Series) -> float:
        """Средняя абсолютная ошибка"""
        return (y_true - y_pred).abs().mean()


    @staticmethod
    def mse(y_true: pd.Series, y_pred: pd.Series) -> float:
        """Среднеквадратичная ошибка"""
        return ((y_true - y_pred)**2).mean()


    @staticmethod
    def rmse(y_true: pd.Series, y_pred: pd.Series) -> float:
        """Квадратный корень из среднеквадратичной ошибки."""
        return np.sqrt(((y_true - y_pred)**2).mean())
class MyLogReg():
    """
    Линейная регрессия

    Гиперпараметры:
    ***************
    n_iter : int, optional
        Количество шагов градиентного спуска (default: 100)
    learning_rate : Union[float, Callable], optional
        Коэффициент скорости обучения градиентного спуска.
        Если на вход пришла lambda-функция, то learning_rate вычисляется 
        на каждом шаге на основе переданной функцией (default: 0.01)
    weights : np.ndarray, optional
        Веса модели (default: None)
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: mae, mse, rmse, mape, r2 (default: None_
    reg : str, optional
        Вид регуляризации. Принимает одно из следующих значений: l1, l2, elasticnet (default: None)
    l1_coef : float, optional
        Коэффициент L1 регуляризации. Принимает значения от 0.0 до 1.0 (default: 0)
    l2_coef : float, optional
        Коэффициент L2 регуляризации. Принимает значения от 0.0 до 1.0 (default: 0)
    sgd_sample : Union[int, float], optional
        Количество образцов, которое будет использоваться на каждой итерации обучения.
        Может принимать целые числа, либо дробные от 0.0 до 1.0 (default: None)
    random_state : int, optional
        Сид для воспроизводимости результата (default: 42)
    """

    def __init__(self,
                n_iter: int = 100,
                learning_rate: float = 0.01,
                weights: np.ndarray = None,
                metric: str = None,
                reg: str = None,
                l1_coef: float = 0,
                l2_coef: float = 0,
                sgd_sample: float = None,
                random_state: int = 42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.metric = metric
        self.score = None
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        self.random_state = random_state


    def __str__(self):
        params = [f'{k}={v}' for k,v in self.__dict__.items()]
        return 'MyLogReg class: ' + ', '.join(params)


    __repr__ = __str__



    def sigmoid(self, value):

        return 1 / (1 + np.exp(-value))
    

    def loss(self, y_true: pd.Series, y_pred: pd.Series) -> float:
        """
        Подсчет значения функции потерь (MSE) 
        """
        eps = 1e-15
        loss = ((y_true * np.log(y_pred + eps)) + (1-y_true) * np.log(1 - y_pred + eps)).mean()
        if self.reg == 'l1':
            loss += self.l1_coef * np.abs(self.weights).sum()
        elif self.reg == 'l2':
            loss += self.l2_coef * np.square(self.weights).sum()
        elif self.reg == 'elasticnet':
            loss += self.l1_coef * np.abs(self.weights).sum() + self.l2_coef * np.square(self.weights).sum()
        return loss


    def gradient(self, X: pd.DataFrame, y_true: pd.Series, y_pred: np.ndarray, batch_idx: list) -> float:
        """
        Подсчет градиента для очередного приближения с учетом регуляризации
        """
        y_true = y_true.iloc[batch_idx]
        y_pred = y_pred[batch_idx]
        X = X.iloc[batch_idx]

        gradient = 1 / len(y_true) * np.dot((y_pred - y_true), X)

        if self.reg == 'l1':
            gradient += self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':
            gradient += self.l2_coef * 2 * self.weights
        elif self.reg == 'elasticnet':
            gradient += self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights 

        return gradient


    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: int = False):
        """
        Обучение линейной регрессии

        Parameters
        ----------
        X : pd.DataFrame
            Все фичи
        y : pd.Series
            Целевая переменная
        verbose : int, optional
            Указывает через сколько итераций градиентного спуска будет выводиться лог
        """
        #random.seed(self.random_state)
        features = X.copy()
        features.insert(loc=0, column='x0', value=1)
        self.weights = np.ones(features.shape[1])

        for i in range(1, self.n_iter + 1):
            #if isinstance(self.sgd_sample, int):
                #batch_idx = random.sample(range(features.shape[0]), self.sgd_sample)
            #elif isinstance(self.sgd_sample, float):
                #batch_idx = random.sample(range(features.shape[0]), int(features.shape[0] * self.sgd_sample))
            #else:
            batch_idx = list(range(features.shape[0]))
            
            y_pred = self.sigmoid(np.dot(features, self.weights))
            loss = self.loss(y, y_pred)
            gradient = self.gradient(features, y, y_pred, batch_idx)

            if callable(self.learning_rate):
                self.weights -= self.learning_rate(i) * gradient
            else:
                self.weights -= self.learning_rate * gradient
            #self.score = MyLogReg.get_metric_score(y, self.predict(X), self.metric)

            if verbose:
                if (i == 1) or (i % verbose == 0):
                    log = f'{i} | loss: {loss}'
                    #if self.metric:
                        #log += f' | metric: {MyLogReg.get_metric_score(y, y_pred, self.metric)}'
                    print(log)


    def predict(self, X: pd.DataFrame):
        """
        Выдача предсказаний моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        features = X.copy()
        features.insert(loc=0, column='x0', value=1)
        y_pred = np.dot(features, self.weights)
        return y_pred


    def get_coef(self):
        return self.weights[1:]


    def get_best_score(self):
        return self.score
        """Коэффициент детерминации"""
        return 1 - ((y_true - y_pred)**2).mean() / ((y_true - y_true.mean())**2).mean()


    @staticmethod
    def mape(y_true: pd.Series, y_pred: pd.Series) -> float:
        """Средняя абсолютная ошибка в процентах """
        return 100 * ((y_true - y_pred) / y_true).abs().mean()


    @staticmethod
    def get_metric_score(y_true: pd.Series, y_pred: pd.Series, metric: str):
        """ 
        Подсчет выбранной метрики
        """
        if metric == 'mae':
            return MyLineReg.mae(y_true, y_pred)
        elif metric == 'mse':
            return MyLineReg.mse(y_true, y_pred)
        elif metric == 'rmse':
            return MyLineReg.rmse(y_true, y_pred)
        elif metric == 'r2':
            return MyLineReg.r2(y_true, y_pred)
        elif metric == 'mape':
            return MyLineReg.mape(y_true, y_pred)
        else:
            ValueError("Wrong metric name")



    def sigmoid(self, value):

        return 1 / (1 + np.exp(-value))
    

    def loss(self, y_true: pd.Series, y_pred: pd.Series) -> float:
        """
        Подсчет значения функции потерь (MSE) 
        """
        eps = 1e-15
        loss = ((y_true * np.log(y_pred + eps)) + (1-y_true) * np.log(1 - y_pred + eps)).mean()
        if self.reg == 'l1':
            loss += self.l1_coef * np.abs(self.weights).sum()
        elif self.reg == 'l2':
            loss += self.l2_coef * np.square(self.weights).sum()
        elif self.reg == 'elasticnet':
            loss += self.l1_coef * np.abs(self.weights).sum() + self.l2_coef * np.square(self.weights).sum()
        return loss


    def gradient(self, X: pd.DataFrame, y_true: pd.Series, y_pred: np.ndarray, batch_idx: list) -> float:
        """
        Подсчет градиента для очередного приближения с учетом регуляризации
        """
        y_true = y_true.iloc[batch_idx]
        y_pred = y_pred[batch_idx]
        X = X.iloc[batch_idx]

        gradient = 1 / len(y_true) * np.dot((y_pred - y_true), X)

        if self.reg == 'l1':
            gradient += self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':
            gradient += self.l2_coef * 2 * self.weights
        elif self.reg == 'elasticnet':
            gradient += self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights 

        return gradient


    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: int = False):
        """
        Обучение линейной регрессии

        Parameters
        ----------
        X : pd.DataFrame
            Все фичи
        y : pd.Series
            Целевая переменная
        verbose : int, optional
            Указывает через сколько итераций градиентного спуска будет выводиться лог
        """
        random.seed(self.random_state)
        features = X.copy()
        features.insert(loc=0, column='x0', value=1)
        self.weights = np.ones(features.shape[1])

        for i in range(1, self.n_iter + 1):
            if isinstance(self.sgd_sample, int):
                batch_idx = random.sample(range(features.shape[0]), self.sgd_sample)
            elif isinstance(self.sgd_sample, float):
                batch_idx = random.sample(range(features.shape[0]), int(features.shape[0] * self.sgd_sample))
            else:
                batch_idx = list(range(features.shape[0]))
            
            y_pred = self.sigmoid(np.dot(features, self.weights))
            loss = self.loss(y, y_pred)
            gradient = self.gradient(features, y, y_pred, batch_idx)

            if callable(self.learning_rate):
                self.weights -= self.learning_rate(i) * gradient
            else:
                self.weights -= self.learning_rate * gradient
            self.score = MyLineReg.get_metric_score(y, self.predict(X), self.metric)

            if verbose:
                if (i == 1) or (i % verbose == 0):
                    log = f'{i} | loss: {loss}'
                    if self.metric:
                        log += f' | metric: {MyLineReg.get_metric_score(y, y_pred, self.metric)}'
                    print(log)


    def predict(self, X: pd.DataFrame):
        """
        Выдача предсказаний моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        features = X.copy()
        features.insert(loc=0, column='x0', value=1)
        y_pred = np.dot(features, self.weights)
        return y_pred


    def get_coef(self):
        return self.weights[1:]


    def get_best_score(self):
        return self.score

In [237]:
class MyLogReg():
    """
    Линейная регрессия

    Гиперпараметры:
    ***************
    n_iter : int, optional
        Количество шагов градиентного спуска (default: 100)
    learning_rate : Union[float, Callable], optional
        Коэффициент скорости обучения градиентного спуска.
        Если на вход пришла lambda-функция, то learning_rate вычисляется 
        на каждом шаге на основе переданной функцией (default: 0.01)
    weights : np.ndarray, optional
        Веса модели (default: None)
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: mae, mse, rmse, mape, r2 (default: None_
    reg : str, optional
        Вид регуляризации. Принимает одно из следующих значений: l1, l2, elasticnet (default: None)
    l1_coef : float, optional
        Коэффициент L1 регуляризации. Принимает значения от 0.0 до 1.0 (default: 0)
    l2_coef : float, optional
        Коэффициент L2 регуляризации. Принимает значения от 0.0 до 1.0 (default: 0)
    sgd_sample : Union[int, float], optional
        Количество образцов, которое будет использоваться на каждой итерации обучения.
        Может принимать целые числа, либо дробные от 0.0 до 1.0 (default: None)
    random_state : int, optional
        Сид для воспроизводимости результата (default: 42)
    """

    def __init__(self,
                n_iter: int = 100,
                learning_rate: float = 0.01,
                weights: np.ndarray = None,
                metric: str = None,
                reg: str = None,
                l1_coef: float = 0,
                l2_coef: float = 0,
                sgd_sample: float = None,
                random_state: int = 42):
        #self.verbose = verbose
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.metric = metric
        self.score = None
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        self.random_state = random_state


    def __str__(self):
        params = [f'{k}={v}' for k,v in self.__dict__.items()]
        return 'MyLogReg class: ' + ', '.join(params)


    __repr__ = __str__


    @staticmethod
    def accuracy(tp, tn, fp, fn) -> float:
        """"""
        return (tp + tn) / (tp + tn + fp + fn) 


    @staticmethod
    def precision(tp, fp) -> float:
        """"""
        return tp / (tp + fp) 


    @staticmethod
    def recall(tp, fn) -> float:
        """"""
        return tp / (tp + fn)


    @staticmethod
    def f1(precision, recall) -> float:
        """"""
        return 2 * (precision * recall) / (precision + recall)


    @staticmethod
    def roc_auc(y_true: pd.Series, y_pred: pd.Series, y_pred_proba: pd.Series) -> float:
        """"""
        y_pred_proba = pd.DataFrame(y_pred_proba.round(10))
        df = pd.concat([y_pred_proba, y_true], axis=1, names=[0,1])
        print(df)
        df = df.sort_values(by=0, ascending=False)

        positives = df[df[1] == 1]
        negatives = df[df[1] == 0]

        total = 0
        for current_score in negatives[0]:
            score_higher = (positives[0] > current_score).sum()
            score_equal = (positives[0] == current_score).sum()
            total += score_higher + 0.5 * score_equal

        return total / (positives.shape[0] * negatives.shape[0])


    @staticmethod
    def get_metric_score(y_true: pd.Series, y_pred: pd.Series, y_pred_proba: pd.Series, metric: str):
        """ 
        Подсчет выбранной метрики
        """

        negative = 0
        positive = 1

        tp = np.sum(np.logical_and(y_pred == positive, y_true == positive))
        tn = np.sum(np.logical_and(y_pred == negative, y_true == negative))
        fp = np.sum(np.logical_and(y_pred == positive, y_true == negative))
        fn = np.sum(np.logical_and(y_pred == negative, y_true == positive))

        if metric == 'accuracy':
            return MyLogReg.accuracy(tp, tn, fp, fn)
        elif metric == 'precision':
            return MyLogReg.precision(tp, fp)
        elif metric == 'recall':
            return MyLogReg.recall(tp, fn)
        elif metric == 'f1':
            return MyLogReg.f1(MyLogReg.precision(tp, fp), MyLogReg.recall(tp, fn))
        elif metric == 'roc_auc':
            return MyLogReg.roc_auc(y_true, y_pred, y_pred_proba)
        else:
            ValueError("Wrong metric name")



    def sigmoid(self, value):

        return 1 / (1 + np.exp(-value))
    

    def loss(self, y_true: pd.Series, y_pred: pd.Series) -> float:
        """
        Подсчет значения функции потерь (MSE) 
        """
        eps = 1e-15
        loss = ((y_true * np.log(y_pred + eps)) + (1-y_true) * np.log(1 - y_pred + eps)).mean()
        if self.reg == 'l1':
            loss += self.l1_coef * np.abs(self.weights).sum()
        elif self.reg == 'l2':
            loss += self.l2_coef * np.square(self.weights).sum()
        elif self.reg == 'elasticnet':
            loss += self.l1_coef * np.abs(self.weights).sum() + self.l2_coef * np.square(self.weights).sum()
        return loss


    def gradient(self, X: pd.DataFrame, y_true: pd.Series, y_pred: np.ndarray, batch_idx: list) -> float:
        """
        Подсчет градиента для очередного приближения с учетом регуляризации
        """
        y_true = y_true.iloc[batch_idx]
        y_pred = y_pred[batch_idx]
        X = X.iloc[batch_idx]

        gradient = 1 / len(y_true) * np.dot((y_pred - y_true), X)

        if self.reg == 'l1':
            gradient += self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':
            gradient += self.l2_coef * 2 * self.weights
        elif self.reg == 'elasticnet':
            gradient += self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights 

        return gradient


    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: int = True):
        """
        Обучение линейной регрессии

        Parameters
        ----------
        X : pd.DataFrame
            Все фичи
        y : pd.Series
            Целевая переменная
        verbose : int, optional
            Указывает через сколько итераций градиентного спуска будет выводиться лог
        """
        #random.seed(self.random_state)
        features = X.copy()
        features.insert(loc=0, column='x0', value=1)
        self.weights = np.ones(features.shape[1])

        for i in range(1, self.n_iter + 1):
            #if isinstance(self.sgd_sample, int):
                #batch_idx = random.sample(range(features.shape[0]), self.sgd_sample)
            #elif isinstance(self.sgd_sample, float):
                #batch_idx = random.sample(range(features.shape[0]), int(features.shape[0] * self.sgd_sample))
            #else:
            batch_idx = list(range(features.shape[0]))
            
            y_pred_proba = self.sigmoid(np.dot(features, self.weights))
            y_pred = (self.predict_proba(X) > 0.5).astype(int)
            loss = self.loss(y, y_pred_proba)
            gradient = self.gradient(features, y, y_pred_proba, batch_idx)

            if callable(self.learning_rate):
                self.weights -= self.learning_rate(i) * gradient
            else:
                self.weights -= self.learning_rate * gradient
            #self.score = MyLogReg.get_metric_score(y, self.predict(X), self.predict_proba(X), self.metric)

            if verbose:
                if (i == 1) or (i % verbose == 0):
                    log = f'{i} | loss: {loss}'
                    if self.metric:
                        log += f' | metric: {MyLogReg.get_metric_score(y, y_pred, y_pred_proba, self.metric)}'
                    print(log)
        self.score = MyLogReg.get_metric_score(y, y_pred, y_pred_proba, self.metric)
    def predict_proba(self, X: pd.DataFrame):
        """
        Выдача предсказаний моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        features = X.copy()
        features.insert(loc=0, column='x0', value=1)
        y_pred = self.sigmoid(np.dot(features, self.weights))
        return y_pred


    def predict(self, X: pd.DataFrame):
        """
        Выдача предсказаний моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        return (self.predict_proba(X) > 0.5).astype(int)
    

    def get_coef(self):
        return self.weights[1:]


    def get_best_score(self):
        return self.score



In [63]:
import numpy as np

arr = np.array([41, 42, 43, 44])

x = [True, False, True, False]

newarr = arr[x]

print(newarr)

[41 43]


In [215]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=14, n_informative=10, noise=15, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [238]:
model = MyLogReg(metric='roc_auc')
model.fit(X,y,verbose=5)
predictions = model.predict(X)
prdictions_proba = model.predict_proba(X)

            0           0
0    0.572953  -48.005272
1    0.711586  145.801614
2    0.951644  -49.114775
3    0.995568   24.902238
4    0.232790 -152.611643
..        ...         ...
995  0.905670   57.484793
996  0.494608   19.204280
997  0.990122   95.095216
998  0.039900 -107.973750
999  0.999148  200.616716

[1000 rows x 2 columns]


ValueError: The column label '0' is not unique.

In [ ]:
predictions.sum()

502

In [133]:
y_pred_proba = np.array([0.86, 0.51, 0.78, 0.6, 0.6, 0.91,0.55, 0.46, 0.42])
y_pred = np.array([1,0,0,1,0,1,0,0,0])

In [137]:
y_pred_proba.round(10).argsort()
y_pred[idx_sort[::-1]]

array([0, 0, 1, 0, 1, 0, 0, 0, 1])

In [144]:
output_array = np.stack((y_pred, y_pred_proba), axis=1)

output_array=output_array[output_array[:, 1].argsort()][::-1]

output_array

array([[1.  , 0.91],
       [0.  , 0.86],
       [0.  , 0.78],
       [0.  , 0.6 ],
       [1.  , 0.6 ],
       [1.  , 0.55],
       [0.  , 0.51],
       [0.  , 0.46],
       [0.  , 0.42]])

In [145]:
y_pred = np.array([1,0,0,1,0,1,0,0,0])
y_pred_proba = np.array([0.91, 0.86, 0.78, 0.6, 0.6, 0.55,0.51,0.46,0.42])

In [95]:
y_pred[:]

7

In [93]:
P = sum(y_pred)
N = len(y_pred) - sum(y_pred)
positive_count = 0
indicators = 0
# list of positive values
#print(y_pred_proba)
for score in -np.sort(-y_pred_proba.round(10)):
    if score <= 0.5:
        positive_count = len([x for x in -np.sort(-positive_list.round(10)) if x > score])
        # number of positive classes with same score
        counts = len(positive_list[positive_list==score]) / 2
        indicators += positive_count + counts
roc_auc = (1 / P * N) * indicators
#return (1 / P * N) * indicators
print (roc_auc, indicators)

23.333333333333332 14.0


In [146]:
#sorted_list = -np.sort(-y_pred_proba.round(10))

sorted_list= np.stack((y_pred, y_pred_proba.round(10)), axis=1)

sorted_list=sorted_list[sorted_list[:, 1].argsort()][::-1]

positive_list = sorted_list[y_pred == 1]
for score, clas in zip(sorted_list, y_pred,):
    if clas == 0:
        print(len(positive_list[positive_list==score]) / 2)
counts

0.5
0.5
1.5
0.5
0.5
0.5


0.0

In [ ]:
y_pred = np.array([1,0,0,1,0,1,0,0,0])
y_pred_proba = np.array([0.91, 0.86, 0.78, 0.6, 0.6, 0.55,0.51,0.46,0.42])

In [180]:
y_pred = np.array([1,0,0,1,0,1,0,0,0])
y_pred_proba = np.array([0.77, 0.86, 0.78, 0.6, 0.6, 0.55,0.91,0.46,0.42])

In [192]:
P = sum(y_pred)
N = len(y_pred) - sum(y_pred)
positive_count = 0
indicators = 0
sorted_list= np.stack((y_pred, y_pred_proba.round(10)), axis=1)
sorted_list=sorted_list[sorted_list[:, 1].argsort()][::-1]
for elem in sorted_list:
    if elem[0] == 0:
        # number of positive classes before
        positive_count = len([pos for pos in sorted_list if pos[0] == 1 and pos[1]>elem[1]])
        # number of positive classes with same score
        counts = len([pos for pos in sorted_list if pos[0] == 1 and pos[1]==elem[1]]) / 2
        indicators += positive_count + counts
        print(indicators, counts)
roc_auc =  indicators / (P * N)

0.0 0.0
0.0 0.0
0.0 0.0
1.5 0.5
4.5 0.0
7.5 0.0


In [182]:
[pos for pos in sorted_list if pos[0] == 1 and pos[1]>elem[1]]

[array([1.  , 0.77]), array([1. , 0.6]), array([1.  , 0.55])]

In [183]:
roc_auc

0.4166666666666667